In [21]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [22]:
df=pd.read_csv("/Users/ravina/Desktop/CustomerChurnPrediction/Customer-Churn-Records.csv")

In [23]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


In [24]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname','Complain'], axis=1)

In [54]:
df.to_csv('customerchurn_with_necessary_features.csv',index=False)

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix


#df = pd.get_dummies(df, columns=['Geography', 'Gender', 'Card Type'],dtype=int,drop_first=True)

# Remove irrelevant columns,top five columns ['Complain', 'Age', 'NumOfProducts', 'IsActiveMember', 'Balance']
#df = df.drop(['RowNumber', 'CustomerId', 'Surname',"HasCrCard","Satisfaction Score","Geography","Gender","Card Type",], axis=1)
#df = df.drop(['RowNumber', 'CustomerId', 'Surname',"HasCrCard","Satisfaction Score","Geography","Gender","Card Type","CreditScore","Tenure","EstimatedSalary","Point Earned","Complain"], axis=1)
#df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Split the dataset into features (X) and target variable (y)
df = pd.get_dummies(df, columns=['Geography', 'Gender', 'Card Type'],dtype=int,drop_first=True)

#X = df.drop(columns=['Geography', 'Gender', 'Card Type','Exited'],axis=1)





In [26]:
Y=df.drop(['Exited'],axis=1)
X = Y.values
y = df['Exited']
print(Y.columns)

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Satisfaction Score',
       'Point Earned', 'Geography_Germany', 'Geography_Spain', 'Gender_Male',
       'Card Type_GOLD', 'Card Type_PLATINUM', 'Card Type_SILVER'],
      dtype='object')


In [27]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_standard = scaler.fit_transform(X)
# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X_standard, y, test_size=0.25, random_state=42)


In [28]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_standard, y, test_size=0.3, random_state=42)


In [29]:
import imblearn
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_train, y_train = ros.fit_resample(X_train, y_train)

In [30]:
X_train.shape

(11092, 16)

# Model Building 
we will develop different classification model and will see which one give the best results. 

In [51]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")

In [32]:
from sklearn.metrics import precision_score,recall_score, accuracy_score, roc_auc_score,f1_score,confusion_matrix

In [52]:
import warnings
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings("ignore")

params = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Kernel type
    'gamma': ['scale', 'auto']  # Kernel coefficient for 'rbf', 'poly', and 'sigmoid'
}


svc = SVC()

svc_model = GridSearchCV(estimator=svc, param_grid=params, n_jobs=-1, scoring='accuracy', cv=5)

# Fit the GridSearchCV object to the data
result = svc_model.fit(X_train, y_train)

# Print the best score and parameters
print("Best: %f using %s" % (result.best_score_, result.best_params_))


Best: 0.910206 using {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}


In [53]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score,roc_auc_score

svc=SVC(C=100,gamma='auto',kernel='rbf')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)

print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_score=roc_auc_score(y_test,y_pred)
print('Recall:', recall)
print('Precision:', precision)

              precision    recall  f1-score   support

           0       0.89      0.87      0.88      2416
           1       0.49      0.54      0.51       584

    accuracy                           0.80      3000
   macro avg       0.69      0.70      0.69      3000
weighted avg       0.81      0.80      0.80      3000

Recall: 0.5376712328767124
Precision: 0.490625


# AdaBoostClassifier


In [33]:
adc=AdaBoostClassifier()
adc.fit(X_train,y_train)
np.mean(cross_val_score(adc,X_train,y_train,scoring='accuracy',cv=10))


0.7710062632515292

In [34]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import GridSearchCV
params=dict()
params['n_estimators']=[50,70,100,150,200]
params['learning_rate'] = [0.0001, 0.001, 0.01, 0.1, 1.0]
params['algorithm'] = ['SAMME']

adc_model=GridSearchCV(estimator=adc,param_grid=params,n_jobs=-1,scoring='accuracy',cv=5)
result=adc_model.fit(X_train,y_train)

#best score and parameters

print("Best: %f using %s" % (result.best_score_, result.best_params_))

Best: 0.768392 using {'algorithm': 'SAMME', 'learning_rate': 1.0, 'n_estimators': 100}


# Observation about Adaboost performance
1. Adaboost model correctly identifies about 75.6% of the customers who actually churned.
2. Aamong all the customers predicted to churn by the model, approximately 48.9% actually churned.
3. The model achieves a moderate level of accuracy, correctly predicting around 79% of the cases across both churn and non-churn classes.

In [35]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

adc=AdaBoostClassifier(n_estimators=200,learning_rate=0.1,)
adc.fit(X_train, y_train)
y_pred = adc.predict(X_test)

print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Recall:', recall)
print('Precision:', precision)

              precision    recall  f1-score   support

           0       0.93      0.81      0.86      2416
           1       0.49      0.76      0.59       584

    accuracy                           0.80      3000
   macro avg       0.71      0.78      0.73      3000
weighted avg       0.84      0.80      0.81      3000

Recall: 0.7551369863013698
Precision: 0.4856828193832599


# DecisionTreeClassifier 

In [36]:

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Define DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=42)

# Define parameters for GridSearchCV
params = {
    'criterion':['gini','entropy'],
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform GridSearchCV
dtc_model = GridSearchCV(estimator=dtc, param_grid=params, n_jobs=-1, scoring='accuracy', cv=5)
result = dtc_model.fit(X_train, y_train)

# Print best score and parameters
print("Best: %f using %s" % (result.best_score_, result.best_params_))


Best: 0.829157 using {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}


# Observation about the performance of Decision Tree classifier
1. Identified approximately 77.6% of the actual positive cases correctly.
2. Out of all the cases the classifier labeled as positive, approximately 48.9.06% were actually positive.
3. Accuracy indicates classifier correctly predicted 79% of the cases in the dataset.


Decision Tree and AdaboostClassifier nearly gave same results.

In [37]:
dtc = DecisionTreeClassifier(criterion='gini',max_depth=10,min_samples_leaf=1,min_samples_split=2,random_state=42)
dtc.fit(X_train, y_train)
y_pred = adc.predict(X_test)

print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Recall:', recall)
print('Precision:', precision)

              precision    recall  f1-score   support

           0       0.93      0.81      0.86      2416
           1       0.49      0.76      0.59       584

    accuracy                           0.80      3000
   macro avg       0.71      0.78      0.73      3000
weighted avg       0.84      0.80      0.81      3000

Recall: 0.7551369863013698
Precision: 0.4856828193832599


# RandomForestClassifier

In [40]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

parameters = {
              "n_estimators":[50,70,80,100,120],
              'max_depth': [3, 4, 5, 6, 7, 9, 11],
              'min_samples_split': [2, 3, 4, 5, 6, 7],
              'criterion': ['entropy', 'gini']
             }

model = RandomForestClassifier()
#gridRandomForest = RandomizedSearchCV(model, parameters, cv = 5, n_jobs = -1)
gridRandomForest = GridSearchCV(model, parameters, cv = 5, n_jobs = -1)
gridRandomForest.fit(X_train, y_train)

print("Best: %f using %s" % (gridRandomForest.best_score_, gridRandomForest.best_params_))
print('n_estimators: ',gridRandomForest.best_estimator_.n_estimators)
print('criterion: ', gridRandomForest.best_estimator_.criterion)
print('Score: ', gridRandomForest.best_score_)
print('Min Split: ', gridRandomForest.best_estimator_.min_samples_split)
print('Max depth: ', gridRandomForest.best_estimator_.max_depth)

Best: 0.900470 using {'criterion': 'gini', 'max_depth': 11, 'min_samples_split': 4, 'n_estimators': 50}
n_estimators:  50
criterion:  gini
Score:  0.9004695898322179
Min Split:  4
Max depth:  11


In [41]:
# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=50, random_state=42,class_weight='balanced',max_depth=11,min_samples_split=4,criterion='gini')

# Fit the model on the training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_classifier.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

# Create a confusion matrix
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)

Accuracy: 0.8396666666666667
Confusion Matrix:
[[2142  274]
 [ 207  377]]


In [42]:
from yellowbrick.classifier import ConfusionMatrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

random_forest = RandomForestClassifier(n_estimators = 70, min_samples_split = 2, max_depth= 11,  criterion = 'gini', random_state = 0)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)

print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Recall:', recall)
print('Precision:', precision)

              precision    recall  f1-score   support

           0       0.91      0.89      0.90      2416
           1       0.59      0.64      0.61       584

    accuracy                           0.84      3000
   macro avg       0.75      0.77      0.76      3000
weighted avg       0.85      0.84      0.85      3000

Recall: 0.6386986301369864
Precision: 0.5892575039494471


In [43]:
feature_imp_random = pd.Series(random_forest.feature_importances_, index = Y.columns).sort_values(ascending = False)
feature_imp_random

Age                   0.286925
NumOfProducts         0.173906
Balance               0.102750
Point Earned          0.074545
EstimatedSalary       0.072354
CreditScore           0.070420
IsActiveMember        0.052949
Geography_Germany     0.040908
Tenure                0.039045
Satisfaction Score    0.025127
Gender_Male           0.018702
Geography_Spain       0.009534
HasCrCard             0.009421
Card Type_PLATINUM    0.008079
Card Type_SILVER      0.007942
Card Type_GOLD        0.007392
dtype: float64

In [44]:
feature_importances = random_forest.feature_importances_

# Select top N features
top_features = Y.columns[np.argsort(feature_importances)[::-1][:10]]
top_features

Index(['Age', 'NumOfProducts', 'Balance', 'Point Earned', 'EstimatedSalary',
       'CreditScore', 'IsActiveMember', 'Geography_Germany', 'Tenure',
       'Satisfaction Score'],
      dtype='object')

In [47]:
parameters = {'learning_rate': [0.01, 0.03, 0.05, 0.1],
              'max_depth': [1 , 2, 3, 4, 5, 6],
              'min_child_weight': [1, 3, 5, 7, 9],
              'subsample': [0.1, 0.3, 0.5, 0.7, 0.9],
              'colsample_bytree': [0.1, 0.3, 0.5, 0.7],
              'n_estimators': [500],
              'gamma': [0.1, 0.3, 0.5, 0.7],
              'reg_alpha': [0.1, 0.3, 0.5],
              'reg_lambda': [0.1, 0.3, 0.5]
             }

model = XGBClassifier()
xgb_grid =RandomizedSearchCV(model,parameters, cv = 2, n_jobs = -1)
xgb_grid.fit(X_train, y_train)

print('Score: ', xgb_grid.best_score_)
print('Params: ', xgb_grid.best_params_)

Score:  0.8405156869816084
Params:  {'subsample': 0.5, 'reg_lambda': 0.3, 'reg_alpha': 0.5, 'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.5}


In [48]:
xgb = XGBClassifier(subsample = 0.5, reg_lambda = 0.3, reg_alpha = 0.5, n_estimators = 500, min_child_weight = 1, max_depth = 3, learning_rate = 0.1, gamma = 0.1, colsample_bytree = 0.5, random_state = 0)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Recall:', recall)
print('Precision:', precision)



              precision    recall  f1-score   support

           0       0.93      0.85      0.89      2416
           1       0.54      0.72      0.61       584

    accuracy                           0.82      3000
   macro avg       0.73      0.78      0.75      3000
weighted avg       0.85      0.82      0.83      3000

Recall: 0.7157534246575342
Precision: 0.5372750642673522


# The accuracy here at XGB is also good, but same problem, the model learned a lot the negative result not the positive one. Therefore let's take random forest model as final model.

In [50]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

features = X
target = y

best_features = SelectKBest(score_func = chi2,k = 'all')
fit = best_features.fit(features,target)

featureScores = pd.DataFrame(data = fit.scores_,index = list(Y.columns),columns = ['Chi Squared Score']) 

featureScores.sort_values(by = 'Chi Squared Score', ascending = False).round(2)

,Chi Squared Score
Balance,7156628.86
EstimatedSalary,51542.37
Age,2299.99
Geography_Germany,225.01
IsActiveMember,118.54
CreditScore,102.91
Gender_Male,51.30
Geography_Spain,20.97
Point Earned,18.02
NumOfProducts,5.01
